In [18]:
import spacy
import numpy
import sense2vec
import random as rn

from numpy import dot
from numpy.linalg import norm
from nltk.corpus import wordnet as wn


nlp = spacy.load('en')
model = sense2vec.load()

In [34]:
def spacy_max_similarity(w1, w2):
    word_1 = nlp.vocab[w1]
    word_2 = nlp.vocab[w2]

    return word_1.similarity(word_2)

In [ ]:
spacy_max_similarity('study', 'call')

In [42]:
def replace_with_word2vec2(word):
    if type(word) != 'str':
        word = str(word)
    num_upbound = 200
    word = nlp.vocab[word]
    print(word.prob)
    # Most similar ones
    #queries = [w for w in word.vocab if w.is_lower == word.is_lower and w.prob >= -15]
    queries = [w for w in word.vocab if w.prob >= -15]
    print(len(queries))
    by_similarity = sorted(queries, key=lambda w: word.similarity(w), reverse=True)
    #print(by_similarity)
    #print([w.lower_ for w in by_similarity[:num_upbound]])
    cand_word_list = [w.lower_ for w in by_similarity[:num_upbound]] # some candidate words
#     sim_list = [w for w in by_similarity[:num_upbound]]
    #print(sim_list)
    print(cand_word_list)
    
replace_with_word2vec2('priority')

-11.034709930419922
53397
['priority', 'priority', 'priorities', 'priorities', 'precedence', 'chore', 'chargeback', 'repercussion', 'wattage', 'payoff', 'competency', 'precaution', 'fallback', 'bazillion', 'relevancy', 'hindrance', 'pushover', 'gratuity', 'bmr', 'pushback', 'replayability', 'likelyhood', 'motivator', 'framerate', 'concern', 'buisness', 'breathalyzer', 'curiousity', 'msrp', 'disincentive', 'task', 'task', 'birthright', 'seperation', 'godsend', 'enabler', 'onus', 'zillion', 'earner', 'timeliness', 'preference', 'irrelevance', 'sufficiency', 'wrongness', 'hashtag', 'cooldown', 'cooldown', 'requisite', 'urgency', 'timezone', 'timezone', 'overdraft', 'equivalency', 'callback', 'waypoint', 'mcat', 'tradeoff', 'potus', 'newb', 'eyesore', 'lsat', 'importance', 'enablers', 'iqs', 'gazillion', 'sysadmin', 'misstep', 'struct', 'leniency', 'crosshair', 'bodyweight', 'pittance', 'ccna', 'qos', 'ssid', 'theist', 'breadwinner', 'midrange', 'uselessness', 'trifecta', 'plausibility', '

In [64]:
synsets = wn.synsets('study', 'n')

list_synsets = []
for i, syn in enumerate(synsets):

    #print('%d. %s' % (i, syn.name()))
    li = (syn.lemma_names())
    list_synsets += (li)
    
    for hyper_syn in syn.hypernyms():
        list_synsets += (hyper_syn.lemma_names())
    
    for hypo_syn in syn.hyponyms():
        list_synsets += (hypo_syn.lemma_names())
    
    for holo_syn in syn.part_holonyms():
        list_synsets += (holo_syn.lemma_names())
        
    for mero_syn in syn.part_meronyms():
        list_synsets += (mero_syn.lemma_names())
    


# Load and Prepare Dataset

In [23]:
read_path = 'data/conll2003/eng.train'
with open(read_path, "r") as ins:
    raw_data = []
    label_data = []
    
    temp_sent = ''
    temp_label = ''
    tkn = False
    
    for line in ins:
        #array.append(line)

        if len(line) == 1:
            raw_data.append(temp_sent)
            label_data.append(temp_label)
            temp_sent = ''
            temp_label = ''
            tkn = False
        else:
            if tkn == True:
                temp_sent += ' '
                temp_label += ' '
                
            temp_sent += line.split()[0] # 단어
            temp_label += line.split()[-1] # NER 라벨
            tkn = True

raw_data: 문장 단위 <br>
label_data: 문장 단위

In [24]:
sent_raw = []
sent_label = []
for i, row in enumerate(raw_data):
    row_nlp = nlp(row)
    if raw_data[i][-1] == '.': # 마지막에 쉼표가 있는 문장들만 선별하자.
        if len(raw_data[i].split()) >= 2: # 길이가 최소 2이상인 문장들마나 선별하자. (ex. .만 있는 문장도 있다)
            temp = []
            for token in row_nlp:
                temp.append(token.pos_) # 각 token의 pos 저장
            
            if 'VERB' in temp: # 문장에 최소 1개이상의 verb가 있어야 한다.
    
                if len([x for x in label_data[i].split() if x != 'O']) != 0: # 엔티티가 하나도 없으면 안된다.
            
                    sent_raw.append(raw_data[i])
                    sent_label.append(label_data[i]) 

sent_raw: 문장 단위 + 정상적인 문장 텍스트 <br>
sent_label: 문장 단위 + 정상적인 문장 텍스트

In [30]:
sent_raw

['EU rejects German call to boycott British lamb .',
 'The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep .',
 "Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer .",
 '" We do n\'t support any such recommendation because we do n\'t see any grounds for it , " the Commission \'s chief spokesman Nikolaus van der Pas told a news briefing .',
 'He said further scientific study was required and if it was found that action was needed it should be taken by the European Union .',
 'He said a proposal last month by EU Farm Commissioner Franz Fischler to ban sheep brains , spleens and spinal cords from the human and animal food chains was a highly specific and precautionary move to protect human health .'

In [25]:
global empty_num_word2vec
global empty_num_sense2vec
global empty_num_wordnet
empty_num_sense2vec = 0

def replace_with_word2vec(word):
    if type(word) != 'str':
        word = str(word)
    num_upbound = 50
    word = nlp.vocab[word]

    # Most similar ones
    queries = [w for w in word.vocab if w.is_lower == word.is_lower and w.prob >= -15]
    by_similarity = sorted(queries, key=lambda w: word.similarity(w), reverse=True)

    #print([w.lower_ for w in by_similarity[:num_upbound]])
    cand_word_list = [w.lower_ for w in by_similarity[:num_upbound]] # some candidate words
    
    # 후보군들 중에서 어떤 것을 교체어로 선정할 것인가?
    
    
    
    return cand_word_list
   
def replace_with_sense2vec(word, pos):
    global empty_num_sense2vec
    
    if type(word) != 'str':
        word = str(word)
    num_upbound = 50
    
    try:
        freq, query_vector = model[word+'|'+pos]
    except KeyError:
        empty_num_sense2vec += 1
        return 'sense2vec is empty!!!'
    
    return model.most_similar(query_vector, n=num_upbound)

def replace_with_wordnet(word, pos):
    if type(word) != 'str':
        word = str(word)   
        
    dog_synsets = wn.synsets(word,pos) # wordnet은 이렇게 POS도 인자로 넣을 수 있다
    for i, syn in enumerate(dog_synsets):
        print('%d. %s' % (i, syn.name()))
        print('alternative names (lemmas): "%s"' % '", "'.join(syn.lemma_names()))
        print('hypErnyms: ', syn.hypernyms())
        print('hypOrnyms: ', syn.hyponyms())
        print('part_holonyms: ', syn.part_holonyms())
        print('part_meronyms: ', syn.part_meronyms()) 
        
        #print('definition: "%s"' % syn.definition())
        #if syn.examples():
        #    print('example usage: "%s"' % '", "'.join(syn.examples()))

In [26]:
def preprocessing_for_spacy(sent_raw):
    
    ### 문장 string 전체 단위
    sent_raw = sent_raw.replace("'ve", 'have')
    
    ### 문장 내에 있는 token 단위
    # nlp output과 sent_raw의 길이가 같도록 하기 위해 전처리 실시
    # ex. nlp()를 거치면, EU-wide와 같은 단어는 EU, -, wide로 3개로 분리된다. 
    splited_sent_raw = sent_raw.split()
    for i, token in enumerate(splited_sent_raw):
        splited_token = token.split('-')
        if not splited_token == 1: # 일반 단어들이 아니라면, result. 'EU-wide', '--'
            if not (splited_token[0] == '' or splited_token[-1] =='km'): # '-'으로만 이뤄진 단어가 아니라면, result.'EU-wide'
                filtered_token = [x for x in splited_token if x != '-'] # '-'를 list에서 삭제 result.['EU', 'wide']
                merged_token = ''.join(filtered_token)
                splited_sent_raw[i] = merged_token
    
    for i, token in enumerate(splited_sent_raw):
        if not len(token) == 1: # . 한개만 있는 token은 제외
            splited_sent_raw[i] = splited_sent_raw[i].replace('.', '')
            splited_sent_raw[i] = splited_sent_raw[i].replace('$', '')
        if token == 'cannot':
            splited_sent_raw[i] = "can"
        if token == 'dont':
            splited_sent_raw[i] = "do"
        if token == "'re":
            splited_sent_raw[i] = "are"
        if token == "**":
            splited_sent_raw[i] = "*"        
        if token == "..." or token == ".." or token == "....":
            splited_sent_raw[i] = "."         
        if token == "'m":
            splited_sent_raw[i] = "am"         
        if token == "'ll":
            splited_sent_raw[i] = "will" 
        if token == "'d":
            splited_sent_raw[i] = "would"
            
        # very specific problem of this task    
        if token == "*Note":
            splited_sent_raw[i] = "Note"   
        if token == "*Name":
            splited_sent_raw[i] = "Name" 
        if token == 'km':
            splited_sent_raw[i] = "miles"             
        if token == '237km':
            splited_sent_raw[i] = "237-km"             
            
            
    return ' '.join(splited_sent_raw)


def generation_via_replace(sent_raw, sent_label):
    # 동사, 명사는 context 정보의 핵심이다 모든 언어가 공통적으로 가지고 있는 특징.    
    
    sent_raw = preprocessing_for_spacy(sent_raw)
    
    sent_nlp = nlp(sent_raw)
    noun_sorted = []
    verb_sorted = []
    
    
    if not len(sent_nlp) == len(sent_label.split()):
        print(len(sent_raw.split()), len(sent_nlp), len(sent_label.split()))
        print(sent_raw)
        print(sent_nlp)
        
        for i, token in enumerate(sent_nlp):
            print(token, sent_raw.split()[i], sent_label.split()[i])
        
    assert(len(sent_nlp) == len(sent_label.split()))
    
    
    
    # 조건에 맞은 Token들 가져오기
    for i, token in enumerate(sent_nlp):
        if sent_label.split()[i] == 'O': # 엔티티가 아닌 경우들만..
            if token.pos_ == 'NOUN' or token.pos_ == 'PROPN':
                pair = (sent_nlp[i], len(list(token.subtree)), i)
                noun_sorted.append(pair) # 명사
            elif token.pos_ == 'VERB':
                pair = (sent_nlp[i], len(list(token.subtree)), i)
                verb_sorted.append(pair) # 동사               
    
    
    if len(noun_sorted) != 0:
        noun_sorted.sort(key=lambda x: x[1], reverse=True)
        replaced_noun = (noun_sorted[0][0], noun_sorted[0][2]) # (token, index) pair 

        # 명사
        print('<<< ', replaced_noun[0], '  >>>')
        print(replace_with_word2vec(replaced_noun[0]))
        print(replace_with_sense2vec(replaced_noun[0], 'NOUN'))
        print(replace_with_wordnet(replaced_noun[0], 'n'))
        print('\n')
    
    if len(verb_sorted) != 0:
        verb_sorted.sort(key=lambda x: x[1], reverse=True)
        replaced_verb = (verb_sorted[0][0], verb_sorted[0][2])

        # 동사
        print('<<< ', replaced_verb[0], '  >>>')
        print(replace_with_word2vec((replaced_verb[0])))
        print(replace_with_sense2vec(replaced_verb[0], 'VERB'))
        print(replace_with_wordnet(replaced_verb[0], 'v'))    
        print('\n')    
    
    


    

    
    
    #return new_sent

## Main

In [28]:
new_sent = []
new_label = []
idx = 0
for i, row in enumerate(sent_raw):
    
    if rn.choice([True, False, False]) == False:
        continue
   
    
    print(sent_raw[i])
    #print(sent_label[i])
    generation_via_replace(sent_raw[i], sent_label[i])
    idx += 1
    print('-------------------------------------------------------------------------')
    if idx==100:
        break
    

EU rejects German call to boycott British lamb .
<<<  call   >>>
['call', 'calls', 'calling', 'grope', 'yearn', 'squeal', 'called', 'misquote', 'quip', 'cower', 'tweet', 'scoff', 'scold', 'crucify', 'wail', 'gloat', 'euthanize', 'cuss', 'patronize', 'poach', 'fantasize', 'whine', 'yawn', 'sneer', 'squirm', 'idolize', 'fax', 'misspell', 'botch', 'twirl', 'grumble', 'scrounge', 'logout', 'lunge', 'rile', 'purr', 'disown', 'chastise', 'consider', 'deride', 'overreact', 'berate', 'retry', 'brag', 'subpoena', 'overdo', 'phoned', 'shrug', 'invite', 'liken']
(['call|NOUN', 'calls|NOUN', 'first_call|NOUN', 'initial_call|NOUN', 'same_call|NOUN', 'original_call|NOUN', 'only_call|NOUN', 'other_calls|NOUN', 'other_call|NOUN', 'next_call|NOUN', 'single_call|NOUN', 'phone_call|NOUN', 'caller|NOUN', 'missed_call|NOUN', 'second_call|NOUN', 'phonecall|NOUN', 'few_calls|NOUN', 'many_calls|NOUN', 'several_calls|NOUN', '_call|NOUN', 'actual_call|NOUN', 'fucking_call|NOUN', 'voice_mail|NOUN', 'dispatch|NOU

In [ ]:
nlp = spacy.load('en')

In [ ]:
test = 'About 350 adventurers from nine countries set out on Saturday to climb , raft , bike and run in a 323-mile ( 517-km ) endurance race through the Canadian wilderness .'

In [ ]:
nlp = nlp(test)

In [ ]:
for token in nlp:
    print(token)

In [35]:
empty_num_sense2vec

98

In [ ]:
for token in nlp:
    print(token)

In [ ]:
temp = [('c', 2), ('a',1), ('c', 3)]

In [ ]:
sorted(temp, key = lambda x: x[1])

In [ ]:
temp.sort(key=lambda x: x[1], reverse=True)

In [ ]:
temp

In [ ]:
doc = nlp(u'I like green eggs and ham.')
for np in doc.noun_chunks:
    print(np.text, np.root.text, np.root.dep_, np.root.head.text)

In [ ]:
cnt = 0
s_cnt = 0
for sent in raw_data:
    cnt += 1
    if list(sent)[-1] == '.':
        s_cnt += 1

In [ ]:
cnt

In [ ]:
s_cnt

# Replace

In [ ]:
### 어떤 것을 교체할 것인가?
# 전략1: 모두 교체
# 전략2: 전치사(closed-class words) 빼고 다 교체
# 전략3: 명사, 동사만 교체
# 전략4: dependency가 높은 명사, 동사만 교체
# 엔티티의 head
# 전략5: root token만 교체

### 어떻게 교체할 것인가?
# word2vec을 사용해서
# wordnet을 사용해서
# sense2vec 사용하면 임베딩에서 태그정보도 같이 넣을 수 있음.

# NER사전을 통해 다른 엔티티로 교체 (NER에 한해서만) - https://explosion.ai/blog/sense2vec-with-spacy
## -> 그러면 엔티티에 대한 패턴을 좀 더 확장할 수 있지 않을까



# Word2vec

## 방법 1

In [ ]:
# #def cosine_similarity(word):


# nasa = parser.vocab['dog']

# # cosine similarity
# cosine = lambda v1, v2: dot(v1, v2) / (norm(v1) * norm(v2))

# # gather all known words, take only the lowercased versions
# allWords = list({w for w in parser.vocab if w.has_vector and w.orth_.islower() and w.lower_ != "nasa"})

# # sort by similarity to NASA
# allWords.sort(key=lambda w: cosine(w.vector, nasa.vector))
# allWords.reverse()
# print("Top 10 most similar words to NASA:")
# for word in allWords[:10]:
#     print(word.orth_)

## 방법 3: sense2vec

In [ ]:
most_similar(nlp.vocab[u'recommendations'])

In [ ]:

freq, query_vector = model["syria|GPE"]
model.most_similar(query_vector, n=5)

In [ ]:
freq, query_vector = model["apple|NOUN"]
model.most_similar(query_vector, n=3)

# Wordnet

In [ ]:
from nltk.corpus import wordnet as wn

# Main

In [ ]:
### MAIN
gen_data = []
for i, row in enumerate(raw_data):
    
    if label_data[i] == 'O': # NER이 아니라면..
    
        new_sent = gen(raw_data[i]):
        
        gen_data.append(new_sent)

    break